In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Load Dataset

In [ ]:
#Test Datasets
from datasets import load_dataset
dataset_test = load_dataset('audiofolder', data_dir='E:/HuggingFace/Test/')
print(dataset_test)

In [ ]:
#Load Datasets
from datasets import load_dataset
dataset_custom = load_dataset('trueorfalse441/Korean_Speech_BigDB')
#dataset_custom = load_dataset("audiofolder", data_dir="E:/HuggingFace/datasets", cache_dir="D:/.cache/datasets/")
dataset_zeroth = load_dataset("Bingsu/zeroth-korean")
dataset_fleurs = load_dataset('google/fleurs', 'ko_kr')

In [ ]:
dataset_custom.push_to_hub("trueorfalse441/Korean_Speech_BigDB")

In [ ]:
print(dataset_custom)
print(dataset_zeroth)
print(dataset_fleurs)

PreProcessing Datasets

In [ ]:
dataset_fleurs = dataset_fleurs.rename_column('transcription', 'text')
dataset_custom = dataset_custom.rename_column("transcription", 'text')

use_cols=['audio', 'text']
dataset_fleurs = dataset_fleurs.remove_columns([col for col in dataset_fleurs.column_names['train'] if col not in use_cols])


In [ ]:
print(dataset_custom)
print(dataset_zeroth)
print(dataset_fleurs)

In [ ]:
# dataset_custom['train'] =  dataset_custom["train"].shuffle(seed=42).select(range(500000))
dataset_custom=dataset_custom['train'].train_test_split(test_size=0.01, shuffle=True)

In [ ]:
from datasets import DatasetDict, IterableDatasetDict
from datasets import concatenate_datasets
from datasets import interleave_datasets
from datasets import Audio

dataset_fleurs = dataset_fleurs.cast_column("audio", Audio(sampling_rate=16000))
dataset_custom = dataset_custom.cast_column("audio", Audio(sampling_rate=16000))
dataset_zeroth = dataset_zeroth.cast_column("audio", Audio(sampling_rate=16000))

data_dict = DatasetDict()

data_dict["train"] = concatenate_datasets(
    [
        dataset_fleurs['train'],
        dataset_zeroth['train'],
        dataset_custom['train'],
    ],
)

data_dict["test"] = concatenate_datasets(
    [
        dataset_fleurs['test'],
        dataset_fleurs['validation'],
        dataset_zeroth['test'],
        dataset_custom['test'],
    ],
)



In [ ]:
data_dict = DatasetDict(data_dict)
data_dict.column_names
print(dataset_custom)
print(data_dict)

In [ ]:
#Load FreatureExtractor
from transformers import WhisperFeatureExtractor
#Load WhisperTokenizer
from transformers import WhisperTokenizer

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-large")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-large", language="Korean", task="transcribe")

In [ ]:
dataset_custom['train'][0]['audio']
dataset_custom

In [ ]:
#Prepare Dataset
def prepare_dataset(batch, feature_extractor=feature_extractor, tokenizer=tokenizer) :
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["text"]).input_ids
    return batch

#Verify Tokenizer
input_str = data_dict['train'][0]['text']
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

#Combine To Create A WhisperProcessor
from transformers import WhisperProcessor
processor = WhisperProcessor.from_pretrained("openai/whisper-large", language="Korean", task="transcribe")

#ReSampling
from datasets import Audio
import multiprocessing
dataset = data_dict.cast_column("audio", Audio(sampling_rate=16000))

dataset = dataset.map(prepare_dataset, 
                            remove_columns=dataset.column_names["train"], num_proc=6,
                            writer_batch_size=6000,
                            fn_kwargs={"feature_extractor": feature_extractor, "tokenizer": tokenizer},
                            
                            #batch_size=100000,                                                                                                                               #batched=True,
                            )

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

In [ ]:
dataset

In [ ]:
dataset.save_to_disk('whisper-1000')

In [ ]:
from datasets import load_from_disk
dataset_test2 = load_from_disk('whisper-1000')

In [ ]:
dataset_test2

#Load FreatureExtractor
from transformers import WhisperFeatureExtractor
#Load WhisperTokenizer
from transformers import WhisperTokenizer

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-large")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-large", language="Korean", task="transcribe")

#Combine To Create A WhisperProcessor
from transformers import WhisperProcessor
processor = WhisperProcessor.from_pretrained("openai/whisper-large", language="Korean", task="transcribe")

dataset=dataset_test2

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

import evaluate
metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

#Load a Pre-Trained Checkpoint
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large")

model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-large-ko-custom-1000h",  # change to a repo name of your choice
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=100,
    max_steps=1500,
    gradient_checkpointing=True,
    #fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

#Train

from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

trainer.train()

Push To hub

In [ ]:
kwargs = {
    "dataset_tags": "ko-speech",
    "dataset": "Speech Text",  # a 'pretty' name for the training dataset
    "dataset_args": "config: ko, split: test",
    "language": "ko",
    "model_name": "Whisper large KR",  # a 'pretty' name for your model
    "finetuned_from": "openai/whisper-large",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}
trainer.push_to_hub(kwargs)

In [ ]:
#reset gpu memory
from numba import cuda
device = cuda.get_current_device()
device.reset()